# 딥러닝 remind 세미나 Homework - Hans.rw

## 필요 툴 라이브러리 설치

In [ ]:
!pip install --upgrade pip
!pip install torch
!pip install torchvision
!pip install numpy
!pip install matplotlib

## Import & 버전 확인 

In [ ]:
!python --version
!nvidia-smi
import os
import torch
import torch.nn.functional as F
import numpy as np
import torchvision
from matplotlib import pyplot as plt
import time
print(torch.__version__)
print(np.__version__)
print(torchvision.__version__)

## Model 선언 - CNN, Linear, 각종 함수 마음껏 섞은 mnist model

In [4]:
class ModelV3(torch.nn.Module):
    def __init__(self):
        super(ModelV3, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = torch.nn.Dropout(0.25)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(9216, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
HOME_DIR = os.path.join(os.getcwd(), os.pardir, os.pardir)
MNIST_AVG = 0.1307
MNIST_STD = 0.3081

seed = '202205281708'
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [6]:
# TODO
# CURL 테스트를 위해서 npy 파일을 받는게 아니라 raw를 받게 해야함.
# 하드코딩 config NO
# model forward 안하고 mode()이렇게 해도 가능함! 
# _device에 따라서 model , data 둘 다 Gpu 로 넘어가야함 (image preprocessing에서 )
class MnistInference():
    def __init__(self):
        #self.MODEL_DIR = os.path.join(HOME_DIR, 'model.pt')
        #self.MODEL_DIR = "./Users/hans.rw/mlops-project-study/2022-06-13/model.pt"
        self.MODEL_DIR = "../model.pt"
        #self.MNIST_DATA_DIR = os.path.join(HOME_DIR, 'mnist_inputs.npy')
        #self.MNIST_DATA_DIR = "./Users/hans.rw/mlops-project-study/2022-06-13/mnist_inputs.npy"
        self.MNIST_DATA_DIR = "../mnist_inputs.npy"
        # TODO: 요 부분들 분포 맞추는 이유 파악하기. normalize
        # normalize 차이? -> 훈련데이터와의 분포를 똑같이 맞춰줘야함. 
        # 훈련데이터와 제일 비슷해야 성능이 잘 나오기 때문?! 
        self._norm_val={"avg":MNIST_AVG, "std":MNIST_STD}
        self._cuda = torch.cuda.is_available()
        self._device = torch.device("cuda" if self._cuda else "cpu")
        self._model = self._get_model()

    def _get_model(self):
        model = torch.load(self.MODEL_DIR)
        self._model.to(self._device)
        model = model.eval()
        return model

    def inference(self, input_image):
        """
        INPUT numpy: mnist image
        OUTPUT int : prediction using model 
        """
        if self._cuda:
            input_image = input_image.to(self._device)

        # TODO: torch.no_grad 무슨의미?
        with torch.no_grad():
            pred = self._model.forward(input_image)
            if self._cuda:
                pre = pred.to("cpu")
            # TODO: 무슨 의미? 1차원으로 바꾸는것?
            pred = pred.argmax(dim=1, keepdim=False)
        return pred
    
    def image_preprocessing(self, data):
        """
        [10, 1, 28, 28]
        """
        data = torch.from_numpy(data)
        # channel dimension
        data = data.unsqueeze(1)
        # 한번 더 하면, batch dimension?
        return data

    def get_mnist_data(self):
        """
        [10, 28, 28]
        """
        data = np.load(self.MNIST_DATA_DIR)
        return data

    # TODO: raw image 값만 Input으로 받아야함. 
    # TODO: 여러장의 input을 받을 때 어떤 식으로 batch 처리를 할 지 로직이 필요함. for 문 돌려서 안함!! batch 처리함 
    # Batch 처리가 훨씬 빠름
    def process(self):
        data = self.get_mnist_data()
        print(data.dtype)
        data = self.image_preprocessing(data)
        print(data.dtype)
        pred = self.inference(data)
        return pred

In [ ]:
mnist_inference = MnistInference()
print(mnist_inference.process())

In [18]:
!ls

mnist_inputs.npy  model.pt  sample_data
